## Pseudonym Detection Under Non-Gaussian Interferences

In [ ]:
import json
import math
import numpy as np
from scipy import signal
import h5py
import matplotlib.pyplot as plt
from matplotlib import rc
import datetime
import scipy.signal as signal
import pandas as pd
import seaborn as sns
import statistics as stats
from scipy.spatial.distance import hamming
import os

rc('xtick', labelsize=14) 
rc('ytick', labelsize=14)

In [ ]:
# Load data from file
def readCom(file_path):
    return np.fromfile(file_path, dtype=np.complex64)

In [ ]:
def get_time_string(timestamp):
    '''
    Helper function to get data and time from timestamp
    INPUT: timestamp
    OUTPUT: data and time. Example: 01-04-2023, 19:50:27
    '''
    date_time = datetime.datetime.fromtimestamp(int(timestamp))
    return date_time.strftime("%m-%d-%Y, %H:%M:%S")

In [ ]:
def JsonLoad(folder, json_file):
    '''
    Load parameters from the saved json file
    INPUT
    ----
        folder: path to the measurement folder. Example: "SHOUT/Results/Shout_meas_01-04-2023_18-50-26"
        json_file: the json file with all the specifications. Example: '/save_iq_w_tx_gold.json'
    OUTPUT
    ----
        samps_per_chip: samples per chip
        wotxrepeat: number of repeating IQ sample collection w/o transmission. Used as an input to 
        traverse_dataset() func
        rxrate: sampling rate at the receiver side
    '''
    #config_file = folder+'/'+json_file
    #config_file = ""+"/"+str(folder)+"/save_iq_w_tx_file.json"
    config_dict = json.load(open(json_file))[0]
    nsamps = config_dict['nsamps']
    rxrate = config_dict['rxrate']
    rxfreq = config_dict['rxfreq']
    wotxrepeat = config_dict['wotxrepeat']
    rxrepeat = config_dict['rxrepeat']
    txnodes = config_dict['txclients']
    rxnodes = config_dict['rxclients']

    return rxrepeat, rxrate, txnodes, rxnodes

In [ ]:
def traverse_dataset(meas_folder):
    '''
    Load data from hdf5 format measurement file
    INPUT
    ----
        meas_folder: path to the measurement folder. Example: "SHOUT/Results/Shout_meas_01-04-2023_18-50-26"
    OUTPUT
    ----
        data: Collected IQ samples w/ transmission. It is indexed by the transmitter name
        noise: Collected IQ samples w/o transmission. It is indexed by the transmitter name
        txrxloc: transmitter and receiver names
    '''
    data = {}
    noise = {}
    txrxloc = {}

    dataset = h5py.File(meas_folder + '/measurements.hdf5', "r") #meas_folder
    #print("Dataset meta data:", list(dataset.attrs.items()))
    for cmd in dataset.keys():
        #print("Command:", cmd)
        if cmd == 'saveiq':
            cmd_time = list(dataset[cmd].keys())[0]
           # print("  Timestamp:", get_time_string(cmd_time))
            #print("  Command meta data:", list(dataset[cmd][cmd_time].attrs.items()))
            for rx_gain in dataset[cmd][cmd_time].keys():
               # print("   RX gain:", rx_gain)
                for rx in dataset[cmd][cmd_time][rx_gain].keys():
                    print("")
                    #print("     RX:", rx)
                    #print("       Measurement items:", list(dataset[cmd][cmd_time][rx_gain][rx].keys()))
        elif cmd == 'saveiq_w_tx':
            cmd_time = list(dataset[cmd].keys())[0]
            #print("  Timestamp:", get_time_string(cmd_time))
            #print("  Command meta data:", list(dataset[cmd][cmd_time].attrs.items()))
            for tx in dataset[cmd][cmd_time].keys():
                #print("   TX:", tx)
                
                if tx == 'wo_tx':
                    for rx_gain in dataset[cmd][cmd_time][tx].keys():
                        #print("       RX gain:", rx_gain)
                       # print(dataset[cmd][cmd_time][tx][rx_gain].keys())
                        for rx in dataset[cmd][cmd_time][tx][rx_gain].keys():
                            #print("         RX:", rx)
                            #print("           Measurement items:", list(dataset[cmd][cmd_time][tx][rx_gain][rx].keys()))
                            repeat = np.shape(dataset[cmd][cmd_time][tx][rx_gain][rx]['rxsamples'])[0]
                            #print("         repeat", repeat)

                            samplesNotx =  dataset[cmd][cmd_time][tx][rx_gain][rx]['rxsamples'][:repeat, :]
                            namelist = rx.split('-')
                            noise[namelist[1]] = samplesNotx
                else:
                    for tx_gain in dataset[cmd][cmd_time][tx].keys():
                        #print("     TX gain:", tx_gain)
                        for rx_gain in dataset[cmd][cmd_time][tx][tx_gain].keys():
                            #print("       RX gain:", rx_gain)
                            #print(dataset[cmd][cmd_time][tx][tx_gain][rx_gain].keys())
                            for rx in dataset[cmd][cmd_time][tx][tx_gain][rx_gain].keys():
                                repeat = np.shape(dataset[cmd][cmd_time][tx][tx_gain][rx_gain][rx]['rxsamples'])[0]
                                #print("         RX:", rx, "; samples shape", np.shape(dataset[cmd][cmd_time][tx][tx_gain][rx_gain][rx]['rxsamples']))
                                #print("         Measurement items:", list(dataset[cmd][cmd_time][tx][tx_gain][rx_gain][rx].keys()))
                                # print("         rxloc", (dataset[cmd][cmd_time][tx][tx_gain][rx_gain][rx]['rxloc'][0]))
                                # peak avg check
                                
                                txrxloc.setdefault(tx, []).extend([rx]*repeat)
                                rxsamples = dataset[cmd][cmd_time][tx][tx_gain][rx_gain][rx]['rxsamples'][:repeat, :]
                                data.setdefault(tx, []).append(np.array(rxsamples))

        else:                       
            print('Unsupported command: ', cmd)

    return data, noise, txrxloc

In [ ]:
# PURPOSE: perform preamble synchronization
#          Uses the (complex-valued) preamble signal. The cross-correlation 
#          of the preamble signal and the received signal (at the time
#          when the preamble is received) should have highest magnitude
#          at the index delay where the preamble approximately starts.  
# INPUT:   rx0: received signal (with a frequency offset)
#          preambleSignal: complex, known, transmitted preamble signal 
# OUTPUT:  lagIndex: the index of rx0 where the preamble signal has highest 
#              cross-correlation
#
def crossCorrelationMax(rx0, preambleSignal):

    # Cross correlate with the preamble to find it in the noisy signal
    lags      = signal.correlation_lags(len(rx0), len(preambleSignal), mode='valid')
    xcorr_out = signal.correlate(rx0, preambleSignal, mode='valid')
    xcorr_mag = np.abs(xcorr_out)
    # Don't let it sync to the end of the packet.
    packetLenSamples = 168000
    maxIndex = np.argmax(xcorr_mag[:len(xcorr_mag)-packetLenSamples])
    lagIndex = lags[maxIndex]

    #print('Max crosscorrelation with preamble at lag ' + str(lagIndex))

#     # Plot the selected signal.
#     plt.figure()
#     fig, subfigs = plt.subplots(2,1)
#     subfigs[0].plot(np.real(rx0), label='Real RX Signal')
#     subfigs[0].plot(np.imag(rx0), label='Imag RX Signal')
#     scale_factor = np.mean(np.abs(rx0))/np.mean(np.abs(preambleSignal))
#     subfigs[0].plot(range(lagIndex, lagIndex + len(preambleSignal)), scale_factor*np.real(preambleSignal), label='Preamble')
#     subfigs[0].legend()
#     subfigs[1].plot(lags, xcorr_mag, label='|X-Correlation|')
#     plt.xlabel('Sample Index', fontsize=14)
#     plt.tight_layout()

    return lagIndex

In [ ]:
def text2bits(message):
    # Convert to characters of '1' and '0' in a vector.
    temp_message = []
    final_message = []
    for each in message:
        temp_message.append(format(ord(each), '07b'))
    for every in temp_message:
        for digit in every:
            final_message.append(int(digit))
    return final_message

In [ ]:
# def Calculate_mean(x):
#     swap = np.zeros(49,)
#     for i in range(49):
#         swap[i] = sum(abs(x[i*N:(i+1)*N])**2)
#         print('Received power:',swap[i])
#     return np.median(swap)

In [ ]:
def binvector2str(binvector):
    #binvector = binvector[0]
    length = len(binvector)
    eps = np.finfo('float').eps
    if abs(length/7 - round(length/7)) > eps:
        print('Length of bit stream must be a multiple of 7 to convert to a string.')
    # Each character requires 7 bits in standard ASCII
    num_characters = round(length/7)
    # Maximum value is first in the vector. Otherwise would use 0:1:length-1
    start = 6
    bin_values = []
    while start >= 0:
        bin_values.append(int(math.pow(2,start)))
        start = start - 1
    bin_values = np.array(bin_values)
    bin_values = np.transpose(bin_values)
    str_out = '' # Initialize character vector
    for i in range(num_characters):
        single_char = binvector[i*7:i*7+7]
        value = 0
        for counter in range(len(single_char)):
            value = value + (int(single_char[counter]) * int(bin_values[counter]))
        str_out += chr(int(value))
    return str_out

In [ ]:
# PURPOSE: create a modulated signal with the defined preamble
# INPUT: A (sqrt value for modulation), N, alpha, Lp (for srrc)
# OUTPUT: modulated preamble signal & srrc pulse
def createPreambleSignal(A, N, alpha, Lp):

    # We defined the preamble as this repeating bit signal:
    preamble     = np.tile([1, 1, 0, 0], 16)

    ###########################################
    ### Signal Generation
    ### INPUT: binary data
    ### OUTPUT: 4-ary data (0..3) values
    data = binary2mary(preamble, 4)

    ###########################################
    ### Modulation
    ### INPUT: data
    ### OUTPUT: modulated values, x
    inputVec   = [0, 1, 2, 3]
    outputVecI = [A, -A, A, -A]
    outputVecQ = [A, A, -A, -A]
    xI         = lut(data, inputVec, outputVecI)
    xQ         = lut(data, inputVec, outputVecQ)
    xI = xI.reshape((1,len(data)))
    xQ = xQ.reshape((1,len(data)))
    ###########################################
    ### Upsample
    ### INPUT: modulated values, x
    ### OUTPUT: modulated values at sampling rate, x_s
    x_s_I = oversample(xI, N)
    x_s_Q = oversample(xQ, N)

    ###########################################
    ### Pulse-shape filter
    ### INPUT: modulated values at sampling rate, x_s
    ### OUTPUT: baseband transmit signal s

    pulse = SRRC(alpha, N, Lp)
    pulse = np.array(pulse)
    pulse = np.reshape(pulse, pulse.size)
    x_s_I = np.reshape(x_s_I, x_s_I.size)
    x_s_Q = np.reshape(x_s_Q, x_s_Q.size)
    s_0_I = np.convolve(x_s_I, pulse, mode='full')
    s_0_Q = np.convolve(x_s_Q, pulse, mode='full')
#     plt.figure()
#     plt.plot(pulse,label='SRRC pulse shape')
#     plt.legend()
#     plt.show()
    
    preamb = s_0_I + 1j*s_0_Q
    #preamb = np.insert(s,0,np.zeros(1024))
    
#     plt.figure()
#     plt.plot(np.real(preamb),label='Real Signal')
#     plt.plot(np.imag(preamb),label='Imag Signal')
#     plt.grid('on')
#     plt.legend()
#     plt.show()
    return preamb, pulse

In [ ]:
# PURPOSE: Convert binary data to M-ary by making groups of log2(M)
#          bits and converting each bit to one M-ary digit.
# INPUT: Binary digit vector, with length as a multiple of log2(M)
# OUTPUT: M-ary digit vector
def binary2mary(data, M):

    log2M   = round(np.log2(M))
    # integer number of bits per group
    if (len(data) % log2M) != 0:
        print('Input to binary2mary must be divisible by log2(m).')
    data.shape = (len(data)//log2M, log2M)
    binaryValuesArray = 2**np.arange(log2M)
    marydata = data.dot(binaryValuesArray)
    return marydata

In [ ]:
# PURPOSE: convert input data stream to signal space values for
#          a particular modulation type (as specified by the inputVec
#          and outputVec).
# INPUT: data (groups of bits)
# OUTPUT: signal space values
def lut(data, inputVec, outputVec):
    if len(inputVec) != len(outputVec):
        print('Input and Output vectors must have identical length')
    # Initialize output
    output = np.zeros(data.shape)
    # For each possible data value
    eps = np.finfo('float').eps
    for i in range(len(inputVec)):
        # Find the indices where data is equal to that input value
        for k in range(len(data)):
            if abs(data[k]-inputVec[i]) < eps:
                # Set those indices in the output to be the appropriate output value.
                output[k] = outputVec[i]
    return output

In [ ]:
# PURPOSE: insert 0's between samples to oversample at OS_Rate
# INPUT: x (data), OS_Rate (how frequently data occurs)
# OUTPUT: x_s (oversampled data)
def oversample(x, OS_Rate):
    # Initialize output
    length = len(x[0])
    x_s = np.zeros((1,length*OS_Rate))
    # Fill in one out of every OS_Rate samples with the input values
    count = 0
    h = 0
    for k in range(len(x_s[0])):
        count = count + 1
        if count == OS_Rate:
            x_s[0][k] = x[0][h]
            count = 0
            h = h + 1
    return x_s

In [ ]:
# PURPOSE: create a square root raised cosine pulse shape
# INPUT: alpha, N, Lp
# OUTPUT: pulse wave array for srrc
def SRRC(alpha, N, Lp):
    # Add epsilon to the n values to avoid numerical problems
    n = np.arange(-N*Lp+ (1e-9), N*Lp+1)
    h = np.zeros(len(n))
    coeff = 1/np.sqrt(N)
    for i, each in enumerate(n):
        sine_term = np.sin(np.pi * each * (1-alpha) / N)
        cosine_term = np.cos(np.pi * each * (1+alpha) / N)
        cosine_coeff = 4 * alpha * each / N
        numerator = sine_term + (cosine_coeff * cosine_term)
        denom_coeff = np.pi * each / N
        denom_part = 1 - cosine_coeff**2
        denominator = denom_coeff * denom_part
        h[i] = coeff * numerator / denominator
    return h

In [ ]:
## Make p-bit decisions by comparing patterns on bit-0 and bit-1
## Trace changes in power with the p-bit and compare it with the known chip pattern.
## This algorithm improves pseudonym detection in the presence of non-Gaussion type interferences

def Matched_Filter_Pseudonym_Detection_Algorithm(x):
    matching_filter =np.array([1,-1,1,-1,1,-1,1,-1,1,-1])
    p_bit = []
    for i in range(28):
        pbit_data = x[i*packet:(i+1)*packet] # slices samples into one p-bit data = 6000 samples
        
        power = []
        for j in range(10):
            chip_data = pbit_data[j*samples:(j+1)*samples] # slice p-bit data into chip data = 600 samples
            power.append(sum(abs(chip_data)**2))
       
        #plt.plot(power)
        #plt.show()
        
        threshold = 0.0
        quantization_level = np.array([1,-1])
        for k in range(9):
            if power[k] > power[k+1]:
                power[k] = quantization_level[0]
            else:
                power[k] = quantization_level[1]
        if power[8] < power[9]:
            power[9] = quantization_level[0]
        else:
            power[9] = quantization_level[1]
            
        pseudonym_power = np.dot(power,matching_filter)#compute the pseudonym power by taking the inner product       

        if pseudonym_power > threshold: # p-bit decision done by comparing p-bit powers with the threshold.
            p_bit.append(1)
        else:
            p_bit.append(0)  
    return np.array(p_bit)

In [ ]:
'''
calculates the distance between the recoded pseudonym bits and the transmitted(true) pseudonym bits.
'''
def Distance(X,Y):
    count = 0
    for i in range(len(X)):
        if X[i]!= Y[i]:
            count +=1
    return count

### RX Below

In [ ]:
A = np.sqrt(9/2)
N = 8
alpha = 0.5
Lp = 6
preambleSignal, pulse = createPreambleSignal(A, N, alpha, Lp)
count_swap = 0
#print("Preamble signal:",preambleSignal)

In [ ]:

'''
Protocol parameters!!!
'''
packet = 6000
samples = 600
pseudonym_len = 28
mess_length = packet*pseudonym_len
repeat = 10
message = 'STOP'

In [ ]:
def Extract_Folders(x):
    r = []
    for root, dirs, files in os.walk(x):
        r.append(dirs)
    return r

In [ ]:
'''
Checks if pseudonyms are decoded correctly in each repeat of shout transmission.
One experiment has 10 repeats.
'''
def Calculate_Eb_No(x):
    # Load parameters from the JSON file which describe what was measured
 
    #folder = x
    jsonfile = "save_iq_w_tx_file.json"
    rxrepeat, samp_rate, txlocs, rxlocs = JsonLoad(x, jsonfile)
    # Load data from the HDF5 file, save IQ sample arrays
    rx_data, rx_noise, txrxloc = traverse_dataset(x)
    samp_rate = 2e6

    txloc = 'cbrssdr1-ustar-comp' # select the TX
    rxloc = 'cbrssdr1-browning-comp' # select the RX
    
    #Calculate SNR
    Noise = rx_noise['browning'][0]  # measure the RX power while the TX is turned off
    noise_power = sum(abs(Noise)**2)/len(Noise)
    
    # initialize error
    
    P_s = 0

    for i in range(repeat):    
        repNum = i
        rx_data[txloc] = np.vstack(rx_data[txloc])
        rxloc_arr = np.array(txrxloc[txloc])
        rx0 = rx_data[txloc][rxloc_arr==rxloc][repNum]
        
        # synchronize pseudonym using preamble signal
        lagIndex = crossCorrelationMax(rx0, preambleSignal)
        start_of_data = lagIndex + len(preambleSignal)
        Rx_signal = rx0[start_of_data:mess_length+start_of_data]

       
        P_s += sum(abs(Rx_signal)**2)/len(Rx_signal)

    signal_power = P_s/repeat   
    return signal_power, noise_power

In [ ]:
'''
Checks if pseudonyms are decoded correctly in each repeat of shout transmission.
One experiment has 10 repeats.
'''

def Calculate_Pseudonym_BER(x):
    # Load parameters from the JSON file which describe what was measured
 
    #folder = x
    jsonfile = "save_iq_w_tx_file.json"
    rxrepeat, samp_rate, txlocs, rxlocs = JsonLoad(x, jsonfile)
    # Load data from the HDF5 file, save IQ sample arrays
    rx_data, rx_noise, txrxloc = traverse_dataset(x)
    samp_rate = 2e6

    txloc = 'cbrssdr1-ustar-comp'
    rxloc = 'cbrssdr1-browning-comp'

    # initialize error
    pseudonym_BER = 0
    P_s = 0
    count = 0
    for i in range(repeat):    
        repNum = i
        rx_data[txloc] = np.vstack(rx_data[txloc])
        rxloc_arr = np.array(txrxloc[txloc])
        rx0 = rx_data[txloc][rxloc_arr==rxloc][repNum]
        
        count += 1
        
        # synchronize pseudonym using preamble signal
        lagIndex = crossCorrelationMax(rx0, preambleSignal)
        start_of_data = lagIndex + len(preambleSignal)
        Rx_signal = rx0[start_of_data:mess_length+start_of_data]

        estimate_pseudonym = Matched_Filter_Pseudonym_Detection_Algorithm(Rx_signal)
        
        if binvector2str(estimate_pseudonym) == 'STOP':
            print('Detected Packet:',binvector2str(estimate_pseudonym))
            print('Pseudonym is correctly detected in ', 85*count, 'ms \n')
            break
        else:
            print('Detected packet:',binvector2str(estimate_pseudonym), '\n')
    
        pseudonym_BER += Distance(text2bits('STOP'),estimate_pseudonym)
            # calculate the average signal power in each repeat
            # signal power = total power - estimated noise power
    return pseudonym_BER

In [ ]:
# Load collected IQ samples from file
def readCom(file_path):
    return np.fromfile(file_path, dtype=np.complex64)

In [ ]:
'''
Computes the average pseudonym error in the experiment.
Experiment data is stored in folders that contain 10 repeats each.
'''
def Pseudonym_Detection_Algorithm(x):
    folders = Extract_Folders(x)[0]
    num_folder = len(folders)
    pseudonym_BER = 0
    signal = 0
    noise = 0
    for i in range(len(folders)):
        s,n = Calculate_Eb_No(folders[i])
        signal += s
        noise += n
    Eb_No = 10*(np.log10(0.5*(signal-noise)/noise))
    
    print('Our Pseudonym message is:', 'STOP')

    print('The Eb/N0 at the RX:',Eb_No)
    print('Data rate: 2Mbps')
    print('Considering only pseudonym transmission latency: \n')
    for i in range(len(folders)):
        print('Experiment No.:',i+1, '\n')
     
        BER = Calculate_Pseudonym_BER(folders[i])
        pseudonym_BER += BER
    
    p_bit_error = pseudonym_BER/(num_folder*repeat*pseudonym_len)
    return p_bit_error 

In [ ]:
# PURPOSE: Detection power change patterns at the received IQ samples and estimates the pseudonym packet.

Pseudonym_Detection_Algorithm("-15dB_data")
